In [5]:
import numpy as np
import torch
import torch.nn as nn

import os

# download the data

In [6]:
# !!curl -O http://www.manythings.org/anki/fra-eng.zip
# !!unzip fra-eng.zip

In [7]:
os.getcwd()

'/home/jeongseobkim/바탕화면/pytorch_reboost/Machine_translator'

In [8]:
!ls

 _about.txt					      fra-eng.zip
'character_level_Machine_translator(seq2seq).ipynb'   fra.txt


In [9]:
!wc -l fra.txt

185583 fra.txt


In [10]:
!head -15 fra.txt

Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
Hi.	Salut.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)
Run!	Cours !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)
Run!	Courez !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)
Run!	Prenez vos jambes à vos cous !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)
Run!	File !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)
Run!	Filez !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077455 (sacredceltic)
Run!	Cours !	CC-BY

In [11]:
!sed -n '9999, 10001p' fra.txt

I owe him $100.	Je lui dois 100 $.	CC-BY 2.0 (France) Attribution: tatoeba.org #260482 (CK) & #127677 (Julien_PDC)
I own this car.	Je possède cette voiture.	CC-BY 2.0 (France) Attribution: tatoeba.org #625545 (rpglover64) & #625632 (U2FS)
I paid in cash.	J'ai payé en espèce.	CC-BY 2.0 (France) Attribution: tatoeba.org #2117578 (erikspen) & #5927003 (Aiji)


In [12]:
!sed -n '154000,154005p' fra.txt

There is a pair of scissors on the desk.	Il y a une paire de ciseaux sur le bureau.	CC-BY 2.0 (France) Attribution: tatoeba.org #20578 (CK) & #335286 (TRANG)
There is almost no water in this bottle.	Il n'y a presque pas d'eau dans cette bouteille.	CC-BY 2.0 (France) Attribution: tatoeba.org #57409 (CK) & #181451 (Romira)
There is an urgent need for peace talks.	On a un besoin urgent de pourparlers de paix.	CC-BY 2.0 (France) Attribution: tatoeba.org #903703 (CK) & #1370659 (sacredceltic)
There is little wine left in the bottle.	Il reste un peu de vin dans la bouteille.	CC-BY 2.0 (France) Attribution: tatoeba.org #34435 (Swift) & #1123872 (sacredceltic)
There is no reason for her to scold you.	Elle n'a pas de raison de te gronder.	CC-BY 2.0 (France) Attribution: tatoeba.org #307974 (CM) & #1829269 (sacredceltic)
There is no reason for her to scold you.	Elle n'a pas de raison de vous gronder.	CC-BY 2.0 (France) Attribution: tatoeba.org #307974 (CM) & #1829270 (sacredceltic)


In [13]:
!tail -1 fra.txt

If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.	Si quelqu'un qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif, cela veut dire qu'il a probablement remarqué quelque chose à propos de votre élocution qui lui a fait prendre conscience que vous n'êtes pas un locuteur natif. En d'autres termes, vous ne parlez pas vraiment comme un locuteur natif.	CC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #955961 (sacredceltic)


In [14]:
!head _about.txt

** Info **

Check for newest version here:
  http://www.manythings.org/anki/
Date of this file:
  2021-01-24

This data is from the sentences_detailed.csv file from tatoeba.org.
http://tatoeba.org/files/downloads/sentences_detailed.csv



# configuration

In [15]:
batch_size = 64
epochs = 100
latent_dim = 256 # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on.

# Path to the data txt file on disk.
data_path = r'fra.txt'

# prepare dataset

In [16]:
# vectorize data

input_texts = []
target_texts = []

input_characters = set()
target_characters = set()

with open(data_path, 'r') as f:
    lines = f.read().split('\n')

In [17]:
# read -> eng / french split
# input_texts = eng, target_texts = french
# lines[: min(num_samples, len(lines)-1)] : num_samples를 lines수보다 높게 잡았을 경우, 에러 방지용 코드
# len(lines)-1 : lines가 Read 과정에서 가장 마지막 줄이 공백으로 포함되어 나옴. 따라서, read_data[-2]까지만 사용해야함
for i, line in enumerate(lines[: min(num_samples, len(lines)-1)]):
    ### line : "I paid in cash. 	J'ai payé en espèce.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)"
    
    try : 
        input_text, target_text, _ = line.split(sep='\t')
    except:
        print(i)
        break
    ### input_text : "I paid in cash."
    ### target_text : "J'ai payé en espèce."

    
    # We use "tab" as the "start sequence" character ("\t" == "BOS")
    # for the targets, and "\n" as "end sequence" character. ("\n" == "EOS")
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    ### input_texts : ['I paid in cash.', '', ..]
    ### target_texts : ["\tJ'ai payé en espèce.\n", '\t+t+\n', ..]

    # word(or sentence) -> characterize!
    # put the char into the set type (중복 방지)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    ### input_characters = {'c', 'a', 's', 'n', 'p', '.', 'i', 'I', 'd', 'h', '-', '-', ..}
    ### target_characters = {'é', "'", 'a', 'J', 's', 'c', 'y', 'n', 'e', 'p', '\t', '.', 'è', '\n', 'i', ' ', ..}
    
                
# set type -> list type (sorted 유지)
# set('d','c','b','a') -> {'a','b','c','d'}
# list(set('d','c','b','a')) -> ['a', 'h', 'd', 'b', 'c']   *정렬 깨짐
input_characters = sorted(list(input_characters))  # input_characters == encoder token
target_characters = sorted(list(target_characters)) # target_charcters == decoder token == '\t'+target_text+'\n'
### input_characters = [' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']
### target_characters = ['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


# ## check for max length target sentence == 57?
# max_val = 0
# max_idx = 0
# max_txt = str()
# for i, txt in enumerate(target_texts):
#     if len(txt) >= max_val:
#         max_txt = txt
#         max_val = len(txt)
#         max_idx = i+1
        
# print(f"max length of target sentence : {max_val}, index of line : {max_idx}",end='\n \n')
# print(max_txt)

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [18]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print(input_token_index, end='\n\n')
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '5': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'K': 30, 'L': 31, 'M': 32, 'N': 33, 'O': 34, 'P': 35, 'Q': 36, 'R': 37, 'S': 38, 'T': 39, 'U': 40, 'V': 41, 'W': 42, 'Y': 43, 'a': 44, 'b': 45, 'c': 46, 'd': 47, 'e': 48, 'f': 49, 'g': 50, 'h': 51, 'i': 52, 'j': 53, 'k': 54, 'l': 55, 'm': 56, 'n': 57, 'o': 58, 'p': 59, 'q': 60, 'r': 61, 's': 62, 't': 63, 'u': 64, 'v': 65, 'w': 66, 'x': 67, 'y': 68, 'z': 69, 'é': 70}

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '5': 17, '8': 18, '9': 19, ':': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'Q': 38, 'R': 39, 'S': 40, 'T': 41

In [19]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    # one-hot vectorization
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    # one-hot vectorization
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character. (because t > 0)
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        
    # padding (space char) -> 
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    # padding (space char) -> [0][0] 에서 2 index가 1로 채워져있다면, padding 된 것.
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [21]:
encoder_input_data[0:1].shape

(1, 15, 71)

In [22]:
encoder_input_data[0][0][23:30]

array([0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [23]:
encoder_input_data[0][1][54:62]

array([0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [24]:
encoder_input_data[0][2][4:14]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [25]:
target_texts[0]

'\tVa !\n'

In [26]:
print(len(target_texts[0]))

6


In [27]:
decoder_input_data[0][:3]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

In [28]:
decoder_target_data[0][:3]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

In [29]:
decoder_target_data[0][-3:-1]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [30]:
decoder_input_data[0][-3:-1]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [44]:
for i in range(59):
    if decoder_target_data[-1][i][1] == 1:
        print(i)
        break

25


In [45]:
# 25 t-step has EOS token
decoder_target_data[-1][25]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [47]:
# from 26 t-step, it has padding token (index=2)
decoder_target_data[-1][26]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [31]:
print(f"|encoder input data| : {encoder_input_data.shape}")
print(f"|decoder input data| : {decoder_input_data.shape}")
print(f"|decoder target data| : {decoder_target_data.shape}")

|encoder input data| : (10000, 15, 71)
|decoder input data| : (10000, 59, 93)
|decoder target data| : (10000, 59, 93)


In [67]:
# To Tensor
encoder_input_data = torch.FloatTensor(encoder_input_data)
decoder_input_data = torch.FloatTensor(decoder_input_data)
decoder_target_data = torch.FloatTensor(decoder_target_data)

# buile the model

In [68]:
class Encoder(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, dropout_p=.2, bidirectional=True):
        
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout_p = dropout_p
        
        self.lstm = nn.LSTM(
            input_size = self.input_size,
            hidden_size = self.hidden_size,
            num_layers = self.num_layers,
            dropout = self.dropout_p,
            batch_first = True,
            bidirectional = self.bidirectional
        )
    
    def forward(self, x):
        # |x| = (10,000(bs), 15(t-step), 71(|vocab|))
        output_enc, _ = self.lstm(x)
        # |output_enc| = (10,000(bs), 1, 2*hidden_size)
        # _ : (hidden_state, cell_state)
        
        return output_enc
    

In [71]:
encoder = Encoder(encoder_input_data.shape[-1], latent_dim, num_layers=3, bidirectional=True)

In [73]:
temp = encoder(encoder_input_data)

In [74]:
temp.shape

torch.Size([10000, 15, 512])

In [75]:
temp[-1].shape

torch.Size([15, 512])

### supplements

#### lines

In [27]:
lines[:3]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)']

In [54]:
lines[-1:-2:-1]

['']

In [53]:
lines[-1:-3:-1]

['',
 "If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.\tSi quelqu'un qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif, cela veut dire qu'il a probablement remarqué quelque chose à propos de votre élocution qui lui a fait prendre conscience que vous n'êtes pas un locuteur natif. En d'autres termes, vous ne parlez pas vraiment comme un locuteur natif.\tCC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #955961 (sacredceltic)"]

In [68]:
lines[-1:-3:-1][1].split(sep='\t')

["If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.",
 "Si quelqu'un qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif, cela veut dire qu'il a probablement remarqué quelque chose à propos de votre élocution qui lui a fait prendre conscience que vous n'êtes pas un locuteur natif. En d'autres termes, vous ne parlez pas vraiment comme un locuteur natif.",
 'CC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #955961 (sacredceltic)']

In [67]:
lines[-1:-3:-1][1].split(sep='\t')[0]

"If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker."

In [66]:
lines[-1:-3:-1][1].split(sep='\t')[1]

"Si quelqu'un qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif, cela veut dire qu'il a probablement remarqué quelque chose à propos de votre élocution qui lui a fait prendre conscience que vous n'êtes pas un locuteur natif. En d'autres termes, vous ne parlez pas vraiment comme un locuteur natif."

#### input_text, target_text

In [11]:
for line in lines[min(num_samples, len(lines)-1)::-1]:
    input_text, target_text, _ = line.split(sep='\t')
    print(input_text)
    print(target_text)
    break

I paid in cash.
J'ai payé en espèce.


#### input_texts, target_texts

In [12]:
for line in lines[min(num_samples, len(lines)-1)::-1]:
    input_text, target_text, _ = line.split(sep='\t')
    # input_text : "I paid in cash."
    # target_text : "J'ai payé en espèce."

    
    # We use "tab" as the "start sequence" character ("\t" == "BOS")
    # for the targets, and "\n" as "end sequence" character. ("\n" == "EOS")
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    print(input_texts[-1::-1])
    print(target_texts[-1::-1])
    break


['I paid in cash.', 'I own this car.', 'I owe him $100.', 'I often travel.', 'I often travel.', 'I often hiccup.', 'I noticed that.', 'I never go out.', 'I needed money.', 'I needed money.', 'I needed money.', 'I need you now.', 'I need you now.', 'I need to stop.', 'I need to know.', 'I need to know.', 'I need to know.', 'I need to know.', 'I need to know.', 'I need to know.', 'I need surgery.', 'I need support.', 'I need it ASAP.', 'I need friends.', 'I need answers.', 'I need a stamp.', 'I need a snack.', 'I need a raise.', 'I need a knife.', 'I need a favor.', 'I need a drink.', 'I need a drink.', 'I need a break.', 'I must see you.', 'I must see you.', 'I must protest.', 'I must protest.', 'I must protest.', 'I must decline.', 'I must buy one.', 'I missed a lot.', 'I miss my wife.', 'I miss college.', 'I miss college.', 'I met him once.', 'I met a friend.', 'I mean no harm.', 'I may be wrong.', 'I may be wrong.', 'I may be wrong.', 'I made muffins.', 'I made cookies.', 'I loved Fr

#### input_characters, target_characters

In [13]:
# read -> eng / french split 
# input_texts = eng, target_texts = french
# lines[: min(num_samples, len(lines)-1)] : num_samples를 lines수보다 높게 잡았을 경우, 에러 방지용 코드
# len(lines)-1 : lines가 Read 과정에서 가장 마지막 줄이 공백으로 포함되어 나옴. 따라서, read_data[-2]까지만 사용해야함
cnt = 0
for line in lines[min(num_samples, len(lines)-1)::-1]:
    cnt+=1
    input_text, target_text, _ = line.split(sep='\t')
    ### input_text : "I paid in cash."
    ### target_text : "J'ai payé en espèce."

    
    # We use "tab" as the "start sequence" character ("\t" == "BOS")
    # for the targets, and "\n" as "end sequence" character. ("\n" == "EOS")
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    ### input_texts : ['I paid in cash.', '', ..]
    ### target_texts : ["\tJ'ai payé en espèce.\n", '\t+t+\n', ..]
    
    # put the char into the set type (중복 방지)
    for char in input_text:
        print(char)
        if char not in input_characters:
            print(char)
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
    print(input_characters)
    print(target_characters)

    if cnt == 4:
        break

I
 
p
a
i
d
 
i
n
 
c
a
s
h
.
[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']
['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']
I
 
o
w
n
 
t
h
i
s
 
c
a
r
.
[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A

In [14]:
for line in lines[: min(num_samples, len(lines)-1)]:
    ### line : "I paid in cash. 	J'ai payé en espèce.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)"
    
    input_text, target_text, _ = line.split(sep='\t')
    ### input_text : "I paid in cash."
    ### target_text : "J'ai payé en espèce."

    
    # We use "tab" as the "start sequence" character ("\t" == "BOS")
    # for the targets, and "\n" as "end sequence" character. ("\n" == "EOS")
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    ### input_texts : ['I paid in cash.', '', ..]
    ### target_texts : ["\tJ'ai payé en espèce.\n", '\t+t+\n', ..]
    
    # put the char into the set type (중복 방지)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    ### input_characters = {'c', 'a', 's', 'n', 'p', '.', 'i', 'I', 'd', 'h', '-', '-', ..}
    ### target_characters = {'é', "'", 'a', 'J', 's', 'c', 'y', 'n', 'e', 'p', '\t', '.', 'è', '\n', 'i', ' ', ..}
    
                
# set type -> list type (sorted 유지)
# set('d','c','b','a') -> {'a','b','c','d'}
# list(set('d','c','b','a')) -> ['a', 'h', 'd', 'b', 'c']   *정렬 깨짐
input_characters = sorted(list(input_characters))  # input_characters == encoder token
target_characters = sorted(list(target_characters)) # target_charcters == decoder token


print(input_characters)
print(target_characters)

[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']
['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


#### EDA

In [15]:
!wc -l fra.txt

185583 fra.txt


In [16]:
print(len(lines))

185584


In [17]:
print("last : " ,lines[-1])
print()
print("lines[-2] : ", lines[-2])

last :  

lines[-2] :  If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.	Si quelqu'un qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif, cela veut dire qu'il a probablement remarqué quelque chose à propos de votre élocution qui lui a fait prendre conscience que vous n'êtes pas un locuteur natif. En d'autres termes, vous ne parlez pas vraiment comme un locuteur natif.	CC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #955961 (sacredceltic)


**Max sequence length for outputs: 57?**


In [151]:
!sed -n '114609,114610p' fra.txt

You'll forget about me someday.	Vous m'oublierez un jour.	CC-BY 2.0 (France) Attribution: tatoeba.org #1602 (CK) & #495963 (sacredceltic)
You'll need a temporary bridge.	Vous avez besoin d'un bridge provisoire.	CC-BY 2.0 (France) Attribution: tatoeba.org #25320 (CM) & #486742 (hortusdei)


In [154]:
len("Vous avez besoin d'un bridge provisoire.")

40

In [157]:
len("Vous avez besoin d'un bridge provisoire.	")

41

In [108]:
!sed -n '64614,64616p' fra.txt

Why do you want to know?	Pourquoi voulez-vous savoir ?	CC-BY 2.0 (France) Attribution: tatoeba.org #2011981 (CK) & #3283203 (Dreamk33)
Why do you wear a watch?	Pourquoi portes-tu une montre ?		CC-BY 2.0 (France) Attribution: tatoeba.org #2645149 (CK) & #3483127 (Lulantis)
Why does he look grumpy?	Pourquoi a-t-il l'air grincheux ?	CC-BY 2.0 (France) Attribution: tatoeba.org #487778 (CM) & #550786 (sacredceltic)


In [143]:
!sed -n '54609,54611p' fra.txt

I'm writing a new book.	Je suis en train d'écrire un nouveau livre.	CC-BY 2.0 (France) Attribution: tatoeba.org #5938921 (CK) & #7862802 (Aiji)
I've already done that.	J'ai déjà fait ça.	CC-BY 2.0 (France) Attribution: tatoeba.org #2358878 (CK) & #1820431 (sacredceltic)
I've already had lunch.	J'ai déjà déjeuné.	CC-BY 2.0 (France) Attribution: tatoeba.org #5850430 (CK) & #5552364 (sacredceltic)


In [ ]:
I'm writing a new book.	Je suis en train d'écrire un nouveau livre.	CC-BY 2.0 (France) Attribution: tatoeba.org #5938921 (CK) & #7862802 (Aiji)
I've already done that.	J'ai déjà fait ça.	CC-BY 2.0 (France) Attribution: tatoeba.org #2358878 (CK) & #1820431 (sacredceltic)
I've already had lunch.	J'ai déjà déjeuné.	CC-BY 2.0 (France) Attribution: tatoeba.org #5850430 (CK) & #5552364 (sacredceltic)
I've analyzed the data.	J'ai analysé les données.	CC-BY 2.0 (France) Attribution: tatoeba.org #2359005 (CK) & #6083198 (Aiji)

In [110]:
!sed -n '14610,14611p' fra.txt

She is not tall.	Elle n'est pas grande.	CC-BY 2.0 (France) Attribution: tatoeba.org #437589 (MikeDee) & #134588 (Julien_PDC)
She is powerful.	Elle est puissante.	CC-BY 2.0 (France) Attribution: tatoeba.org #1578329 (CM) & #1578330 (Julien_PDC)


In [134]:
!sed -n '4603,4608p' fra.txt

I felt naked.	Je me suis senti nue.	CC-BY 2.0 (France) Attribution: tatoeba.org #3494454 (Hybrid) & #3514630 (sacredceltic)
I felt safer.	Je me sentais davantage en sécurité.	CC-BY 2.0 (France) Attribution: tatoeba.org #3821063 (CK) & #4681988 (sacredceltic)
I go jogging.	Je vais courir.		CC-BY 2.0 (France) Attribution: tatoeba.org #881775 (CK) & #1080740 (sacredceltic)
I go to work.	Je pars au boulot.	CC-BY 2.0 (France) Attribution: tatoeba.org #879627 (CM) & #863525 (Aiji)
I go to work.	Je vais travailler.		CC-BY 2.0 (France) Attribution: tatoeba.org #879627 (CM) & #1188039 (sacredceltic)
I go to work.	Je me rends au travail.		CC-BY 2.0 (France) Attribution: tatoeba.org #879627 (CM) & #1188040 (sacredceltic)


#### set -> list -> sort

In [18]:
a = set(['d','c','b','a'])
a.add('h')

In [19]:
a

{'a', 'b', 'c', 'd', 'h'}

In [20]:
list(a)

['h', 'b', 'd', 'c', 'a']

In [21]:
sorted(list(a))

['a', 'b', 'c', 'd', 'h']

In [22]:
b = [1,2,3]
b.append(4)
b

[1, 2, 3, 4]

In [23]:
b[-1]

4

#### size of input data

In [176]:
encoder_input_data.shape

(10000, 15, 71)

In [178]:
encoder_input_data[0].shape

(15, 71)

In [181]:
(encoder_input_data[0] == 0).sum() == 15*71

True

In [182]:
decoder_input_data.shape

(10000, 59, 93)

In [184]:
decoder_target_data.shape

(10000, 59, 93)

In [213]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    print(input_text)
    print(target_text)
    for t, char in enumerate(input_text):
        print()
        print(t)
        print(char)
        encoder_input_data[i, t, input_token_index[char]] = 1.0
        print()
        print(i, t, input_token_index[char])
        print(encoder_input_data[i, t, input_token_index[char]])
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    print()
    print(encoder_input_data[i, t, input_token_index[char]])
    
    
    break
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Go.
	Va !


0
G

0 0 26
1.0

1
o

0 1 58
1.0

2
.

0 2 9
1.0

1.0
